In [2]:
!pip install ibm_watson bs4

     |████████████████████████████████| 368kB 15.8MB/s eta 0:00:01
     |████████████████████████████████| 204kB 31.0MB/s eta 0:00:01
     |████████████████████████████████| 112kB 29.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/73/21/91/61a59270c329be2edc947c69871c25c50bf7dd87ebbfe07cdc
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson bs4 ibm-cloud-sdk-core


In [3]:
import os
from glob import glob
from bs4 import BeautifulSoup
import IPython
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import SpeechToTextV1
from ibm_watson import AssistantV2
from ibm_watson import TextToSpeechV1

In [8]:
recognition_service = SpeechToTextV1(IAMAuthenticator('LZWsmZ5Rl7iOj-ER2Stzd53PjNjGQ4S3dnLMaUgwy41e'))
recognition_service.set_service_url('https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/6ae8d720-ecf9-41d5-b2e9-2832bd216f25')
SPEECH_EXTENSION = "*.wav"
SPEECH_AUDIOTYPE = "audio/wav"

def recognize_audio():
    while len(glob(SPEECH_EXTENSION)) == 0:
        pass
    filename = glob(SPEECH_EXTENSION)[0]
    audio_file = open(filename, "rb")
    print(audio_file)
    os.popen("rm " + filename)
    result = recognition_service.recognize(audio=audio_file, content_type=SPEECH_AUDIOTYPE).get_result()
    return result["results"][0]["alternatives"][0]["transcript"]

In [9]:
assistant = AssistantV2(version='2020-04-01', authenticator=IAMAuthenticator('c_ut-_2vnMKrfAGzJdoZX2AZSAEcPl1bzl2wHiuscV96'))
assistant.set_service_url('https://api.eu-gb.assistant.watson.cloud.ibm.com/instances/06a469b2-90b0-47bd-bb94-b65f1a58c4d7')
ASSISTANT_ID = "883d0939-18cb-4214-b49d-807b6c37a787"
session_id = assistant.create_session(assistant_id=ASSISTANT_ID).get_result()["session_id"]

def message_assistant(text):
    response = assistant.message(assistant_id=ASSISTANT_ID,
                                 session_id=session_id,
                                 input={'message_type': 'text', 'text': text}).get_result()
    print("Message Assistant Block")
    return BeautifulSoup(response["output"]["generic"][0]["text"]).get_text()

In [10]:
synthesis_service = TextToSpeechV1(IAMAuthenticator('El32_a5DOxHp78zYpzgdKul3hOXIIB1-OnnFp4AzX3QD'))
synthesis_service.set_service_url('https://api.eu-gb.text-to-speech.watson.cloud.ibm.com/instances/ad1d2f86-3479-4552-8c33-ab0e08b74721')

def speak_text(text):
    with open('temp.wav', 'wb') as audio_file:
        response = synthesis_service.synthesize(text, accept='audio/wav', voice="en-US_AllisonV3Voice").get_result()
        audio_file.write(response.content)
        print("SPeak It")
    return IPython.display.Audio("temp.wav", autoplay=True)

In [7]:
speak_text(message_assistant(recognize_audio()))